In [3]:
import os
import json
import shutil

In [2]:
from mmdet.datasets import CocoDataset

/opt/conda/envs/det/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
with open('/data/lx_pantos/annotation_231031/all_annotations.json', 'r') as f:
    data = json.load(f)


In [6]:
img_src = '/data/lx_pantos/images/'
img_trg = '/data/lx_pantos/img_231031/'
os.makedirs(img_trg, exist_ok=True)

In [8]:
for img in data['images']:
    # print(img)
    shutil.copy(os.path.join(img_src, img['file_name']), os.path.join(img_trg, img['file_name']))
    # break

{'id': 1, 'width': 369, 'height': 800, 'file_name': '1086500363-1.1.5@2@2@4481336@202525492.jpg', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
{'id': 2, 'width': 600, 'height': 800, 'file_name': '1086622662-1.1.5@1@2@4508824@202556912.jpg', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
{'id': 3, 'width': 600, 'height': 800, 'file_name': '1087831792-1.1.1@1@1@4743145@203092189.jpg', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
{'id': 4, 'width': 600, 'height': 800, 'file_name': '1087937904-1.1.1@1@2@4635134@202834400.jpg', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
{'id': 5, 'width': 600, 'height': 800, 'file_name': '1087961676-1.1.5@1@2@5015281@203624309.jpg', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
{'id': 6, 'width': 800, 'height': 800, 'file_name': '1087999117-2.1.5@1@2@4300954@202169363.jpg', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
{'id

In [1]:
dataset_type = 'CocoDataset'  # Dataset type, this will be used to define the dataset
data_root = '/data/lx_pantos/'  # Root path of data
backend_args = None # Arguments to instantiate the corresponding file backend

train_pipeline = [  # Training data processing pipeline
    dict(type='LoadImageFromFile', backend_args=backend_args),  # First pipeline to load images from file path
    dict(
        type='LoadAnnotations',  # Second pipeline to load annotations for current image
        with_bbox=True,  # Whether to use bounding box, True for detection
        with_mask=True,  # Whether to use instance mask, True for instance segmentation
        poly2mask=True),  # Whether to convert the polygon mask to instance mask, set False for acceleration and to save memory
    dict(
        type='Resize',  # Pipeline that resizes the images and their annotations
        scale=(1333, 800),  # The largest scale of the images
        keep_ratio=True  # Whether to keep the ratio between height and width
        ),
    dict(
        type='RandomFlip',  # Augmentation pipeline that flips the images and their annotations
        prob=0.5),  # The probability to flip
    dict(type='PackDetInputs')  # Pipeline that formats the annotation data and decides which keys in the data should be packed into data_samples
]
test_pipeline = [  # Testing data processing pipeline
    dict(type='LoadImageFromFile', backend_args=backend_args),  # First pipeline to load images from file path
    dict(type='Resize', scale=(1333, 800), keep_ratio=True),  # Pipeline that resizes the images
    dict(
        type='PackDetInputs',  # Pipeline that formats the annotation data and decides which keys in the data should be packed into data_samples
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                   'scale_factor'))
]
train_dataloader = dict(   # Train dataloader config
    batch_size=2,  # Batch size of a single GPU
    num_workers=2,  # Worker to pre-fetch data for each single GPU
    persistent_workers=True,  # If ``True``, the dataloader will not shut down the worker processes after an epoch end, which can accelerate training speed.
    sampler=dict(  # training data sampler
        type='DefaultSampler',  # DefaultSampler which supports both distributed and non-distributed training. Refer to https://mmengine.readthedocs.io/en/latest/api/generated/mmengine.dataset.DefaultSampler.html#mmengine.dataset.DefaultSampler
        shuffle=True),  # randomly shuffle the training data in each epoch
    batch_sampler=dict(type='AspectRatioBatchSampler'),  # Batch sampler for grouping images with similar aspect ratio into a same batch. It can reduce GPU memory cost.
    dataset=dict(  # Train dataset config
        type=dataset_type,
        data_root=data_root,
        # ann_file='annotations/instances_train2017.json',  # Path of annotation file
        ann_file='annotation_231031/0/instances_default.json',  # Path of annotation file
        data_prefix=dict(img='images/'),  # Prefix of image path
        filter_cfg=dict(filter_empty_gt=True, min_size=32),  # Config of filtering images and annotations
        pipeline=train_pipeline,
        backend_args=backend_args))
val_dataloader = dict(  # Validation dataloader config
    batch_size=1,  # Batch size of a single GPU. If batch-size > 1, the extra padding area may influence the performance.
    num_workers=2,  # Worker to pre-fetch data for each single GPU
    persistent_workers=True,  # If ``True``, the dataloader will not shut down the worker processes after an epoch end, which can accelerate training speed.
    drop_last=False,  # Whether to drop the last incomplete batch, if the dataset size is not divisible by the batch size
    sampler=dict(
        type='DefaultSampler',
        shuffle=False),  # not shuffle during validation and testing
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        ann_file='annotation_231031/1/instances_default.json',
        data_prefix=dict(img='images/'),
        test_mode=True,  # Turn on the test mode of the dataset to avoid filtering annotations or images
        pipeline=test_pipeline,
        backend_args=backend_args))
test_dataloader = val_dataloader  # Testing dataloader config

In [2]:
val_evaluator = dict(  # Validation evaluator config
    type='CocoMetric',  # The coco metric used to evaluate AR, AP, and mAP for detection and instance segmentation
    ann_file=data_root + 'annotation_231031/2/instances_default.json',  # Annotation file path
    metric=['bbox', 'segm'],  # Metrics to be evaluated, `bbox` for detection and `segm` for instance segmentation
    format_only=False,
    backend_args=backend_args)
test_evaluator = val_evaluator

In [3]:
# inference on test dataset and
# format the output results for submission.
test_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        ann_file=data_root + 'annotation_231031/2/instances_default.json',
        data_prefix=dict(img='images/'),
        test_mode=True,
        pipeline=test_pipeline))
test_evaluator = dict(
    type='CocoMetric',
    ann_file=data_root + 'annotation_231031/2/instances_default.json',
    metric=['bbox', 'segm'],  # Metrics to be evaluated
    format_only=True,  # Only format and save the results to coco json file
    outfile_prefix='./work_dirs/coco_detection/test')  # The prefix of output json files


In [6]:
from mmengine.utils import get_git_hash
from mmengine.utils.dl_utils import collect_env as collect_base_env

import mmdet

from mmdet.apis import DetInferencer

# Choose to use a config
model_name = 'rtmdet_tiny_8xb32-300e_coco'
# Setup a checkpoint file to load
checkpoint = './checkpoints/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth'

# Set the device to be used for evaluation
device = 'cuda:0'

# Initialize the DetInferencer
inferencer = DetInferencer(model_name, checkpoint, device)

# Use the detector to do inference
img = './demo/demo.jpg'
result = inferencer(img, out_dir='./output')

Loads checkpoint by local backend from path: ./checkpoints/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



/opt/conda/envs/det/lib/python3.8/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/conda/envs/det/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


/opt/conda/envs/det/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an 
upcoming release, it will be required to pass the indexing argument. (Triggered internally at  
/opt/conda/conda-bld/pytorch_1634272068694/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

/root/mmdetection/mmdet/visualization/palette.py:90: UserWarning: __floordiv__ is deprecated, and its behavior will
change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This 
results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, 
rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  scales = 0.5 + (areas - min_area) // (max_area - min_area)

In [4]:
with open('/data/lx_pantos/annotation_231031/3/instances_default.json', 'r') as f:
    data = json.load(f)


In [9]:
imgs = []
for im in data['images']:
    im['id'] = im['file_name']
    


In [10]:
imgs[0]

{'id': 1,
 'width': 800,
 'height': 800,
 'file_name': '1086732260-2.1.5@2@2@6637262@206894716.jpg',
 'license': 0,
 'flickr_url': '',
 'coco_url': '',
 'date_captured': 0}

In [7]:
data['annotations'][0]

{'id': 1,
 'image_id': 1,
 'category_id': 7,
 'segmentation': {'counts': [228223,
   14,
   784,
   23,
   776,
   32,
   767,
   38,
   762,
   44,
   756,
   50,
   750,
   55,
   745,
   63,
   737,
   74,
   726,
   81,
   720,
   89,
   711,
   98,
   702,
   103,
   697,
   115,
   685,
   125,
   675,
   136,
   664,
   149,
   651,
   160,
   640,
   167,
   632,
   183,
   617,
   202,
   597,
   207,
   593,
   208,
   591,
   210,
   590,
   210,
   590,
   210,
   590,
   210,
   590,
   210,
   590,
   211,
   590,
   210,
   590,
   210,
   590,
   210,
   591,
   209,
   591,
   209,
   591,
   209,
   591,
   209,
   590,
   210,
   590,
   210,
   590,
   210,
   589,
   211,
   589,
   211,
   588,
   212,
   588,
   212,
   588,
   212,
   587,
   213,
   588,
   212,
   590,
   210,
   592,
   208,
   598,
   202,
   609,
   191,
   617,
   183,
   623,
   176,
   632,
   168,
   642,
   158,
   654,
   7,
   2,
   137,
   677,
   123,
   688,
   112,
   701,
   98,

In [15]:
cate = [
    {'id': 1, 'name': '압착슬리브_정면', 'supercategory': '정상'},
    {'id': 2, 'name': '압착슬리브_미압착', 'supercategory': '비정상'},
    {'id': 3, 'name': '압착슬리브_측면', 'supercategory': '정상'},
    {'id': 4, 'name': '재사용', 'supercategory': '비정상'},
    {'id': 5, 'name': '구리선', 'supercategory': '비정상'},
    {'id': 6, 'name': '협의_필요', 'supercategory': '비정상'},
    {'id': 7, 'name': '열수축튜브', 'supercategory': '비정상'},
    {'id': 8, 'name': '절연테이프', 'supercategory': '비정상'}
    ]

In [ ]:

data.keys()

In [ ]:
data['annotations'][0]

In [22]:
import os.path as osp
from mmengine.fileio import dump, load
from mmengine.utils import track_iter_progress


def convert_balloon_to_coco(ann_file, out_file, image_prefix):
    data_infos = load(ann_file)

    annotations = []
    images = []
    obj_count = 0
    for idx, v in enumerate(track_iter_progress(data_infos.values())):
    # for idx, v in enumerate(data_infos.items()):
        filename = v['filename']
        img_path = osp.join(image_prefix, filename)
        height, width = mmcv.imread(img_path).shape[:2]

        images.append(
            dict(id=idx, file_name=filename, height=height, width=width))
        
        for _, obj in v['regions'].items():
            assert not obj['region_attributes']
            obj = obj['shape_attributes']
            px = obj['all_points_x']
            py = obj['all_points_y']
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            x_min, y_min, x_max, y_max = (min(px), min(py), max(px), max(py))

            data_anno = dict(
                image_id=idx,
                id=obj_count,
                category_id=0,
                bbox=[x_min, y_min, x_max - x_min, y_max - y_min],
                area=(x_max - x_min) * (y_max - y_min),
                segmentation=[poly],
                iscrowd=0)
            annotations.append(data_anno)
            obj_count += 1

    coco_format_json = dict(
        images=images,
        annotations=annotations,
        categories=[{
            'id': 0,
            'name': 'balloon'
        }])
    dump(coco_format_json, out_file)


In [10]:
data_infos = load('/data/lx_pantos/annotation_231031/0/instances_default.json')

NameError: name 'load' is not defined

In [ ]:
data_infos

In [26]:
for k, v in enumerate(data_infos.items()):
    print(k)
    print(v)
    break

0
('licenses', [{'name': '', 'id': 0, 'url': ''}])


In [23]:
convert_balloon_to_coco(
    ann_file='/data/lx_pantos/annotation_231031/0/instances_default.json',
    out_file='/root/mmdetection/test.json',
    image_prefix='/data/lx_pantos/images/'
    )

TypeError: "tasks" must be a tuple object or a sequence object, but got <class 'dict_values'>

In [7]:
# The new config inherits a base config to highlight the necessary modification
_base_ = '/root/mmdetection/rtmdet_tiny_8xb32-300e_coco.py'

# We also need to change the num_classes in head to match the dataset's annotation
model = dict(
    roi_head=dict(
        bbox_head=dict(num_classes=1), mask_head=dict(num_classes=1)))

# Modify dataset related settings
data_root = '/data/lx_pantos/'
metainfo = {
    'classes':
            (
                '압착슬리브_정면',
                '압착슬리브_미압착',
                '압착슬리브_측면',
                '재사용',
                '구리선',
                '협의_필요',
                '열수축튜브',
                '절연테이프'
                ),
        'palette':
            [
                (220, 20, 60),
                (119, 11, 32),
                (0, 0, 142),
                (0, 0, 230),
                (106, 0, 228),
                (0, 60, 100),
                (0, 80, 100),
                (0, 0, 70)
                ]
    }
train_dataloader = dict(
    batch_size=1,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        ann_file='annotation_231031/0/instances_default.json',
        data_prefix=dict(img='images/')))
val_dataloader = dict(
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        ann_file='annotation_231031/1/instances_default.json',
        data_prefix=dict(img='images/')))
test_dataloader = val_dataloader

# Modify metric related settings
val_evaluator = dict(ann_file=data_root + 'annotation_231031/1/instances_default.json')
test_evaluator = val_evaluator

# We can use the pre-trained Mask RCNN model to obtain higher performance
load_from = 'https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'